In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter('ignore')

In [ ]:
from kaggle_secrets import UserSecretsClient
from google.oauth2.credentials import Credentials
import ee

In [ ]:
#ee.Authenticate()
#!cat ~/.config/earthengine/credentials

In [ ]:
citid19 = pd.read_csv('../input/cdp-unlocking-climate-solutions/Cities/Cities Disclosing/2019_Cities_Disclosing_to_CDP.csv')
citires19 = pd.read_csv('../input/cdp-unlocking-climate-solutions/Cities/Cities Responses/2019_Full_Cities_Dataset.csv')

In [ ]:
area = citires19[(citires19['Question Number']=='0.6') & (citires19['Column Number']==1)][['Account Number','Response Answer']]

In [ ]:
#assume city is a square centered on given (lat, long) with area as declared in the response, and get coordinates of corners
citid19 = citid19[['Account Number','City Location']]
citid19 = pd.merge(citid19, area, on='Account Number')
citid19 = citid19[~(citid19['City Location'].isna()) & ~(citid19['Response Answer'].isna())]
citid19 = citid19.rename(columns={'Response Answer':'Area'})
citid19['Longc'] = citid19['City Location'].apply(lambda x: float(x.split(' ')[1].split('(')[1]))
citid19['Latc'] = citid19['City Location'].apply(lambda x: float(x.split(' ')[2].split(')')[0]))
citid19['d']=np.sqrt(citid19['Area'].astype(float)/2)
citid19['d'] = citid19['d'].apply(lambda x: 50 if x>50 else x)
citid19['lats']= citid19['Latc'] -(1 / 110.54)*citid19['d']
citid19['latn']= citid19['Latc'] +(1 / 110.54)*citid19['d']
citid19['longe']= citid19['Longc'] -(1 / (111.320 * np.cos(citid19['Latc']))) * citid19['d']
citid19['longw']= citid19['Longc'] +(1 / (111.320 * np.cos(citid19['Latc']))) * citid19['d']

In [ ]:
citid19.head()

In [ ]:
mysecret = "mysecret" 
refresh_token = UserSecretsClient().get_secret(mysecret)
credentials = Credentials(
        None,
        refresh_token=refresh_token,
        token_uri=ee.oauth.TOKEN_URI,
        client_id=ee.oauth.CLIENT_ID,
        client_secret=ee.oauth.CLIENT_SECRET,
        scopes=ee.oauth.SCOPES)
ee.Initialize(credentials=credentials)

In [ ]:
column = 'tropospheric_NO2_column_number_density'
dataset = "COPERNICUS/S5P/OFFL/L3_NO2"
begin_date = '2019-01-01'; end_date = '2019-12-31'
s5p = ee.ImageCollection(dataset).filterDate(begin_date, end_date)
ime = s5p.select(column).reduce(ee.Reducer.mean())
ime.getInfo()

In [ ]:
%%time
concs = []
for i in range(len(citid19)):
    row = citid19.iloc[i]
    lats, latn, longe, longw = row['lats'], row['latn'], row['longe'], row['longw']
    aoi = ee.Geometry.Polygon([[[longw, lats],[longw, latn],[longe, latn],[longe, lats]]], None, False)
    no2 = ime.sampleRectangle(aoi)
    arr = no2.getInfo()
    concs.append(np.mean(arr['properties']['tropospheric_NO2_column_number_density_mean']))

In [ ]:
citid19['no2']=concs

In [ ]:
citid19.to_csv('citid19.csv', index=False)